# Introduction

Do not spend too much time trying to get very tiny metrics improvement. Once you have a model with a correct predictive power, you should better spend time explaining your data cleaning & preparation pipeline as well as explanations & visualizations of the results.

The goal is to see your fit with our company culture & engineering needs, spending 50h on an over-complicated approach will not give you bonus points compared to a simple, yet effective, to-the-point solution.

## About the data

The dataset you will be working with is called Emo-DB and can be found [here](http://emodb.bilderbar.info/index-1280.html).

It is a database containing samples of emotional speech in German. It contains samples labeled with one of 7 different emotions: Anger, Boredom, Disgust, Fear, Happiness, Sadness and Neutral. 

Please download the full database and refer to the documentation to understand how the samples are labeled (see "Additional information")
   
The goal of this project is to develop a model which is able to **classify samples of emotional speech**. Feel free to use any available library you would need, but beware of re-using someone else's code without mentionning it!

## Deliverable

The end-goal is to deliver us a zip file containing:
* This report filled with your approach, in the form of an **iPython Notebook**.
* A **5-10 slides PDF file**, containing a technical presentation covering the important aspects of your work
* A Dockerfile which defines a container for the project. The container should handle everything (download the data, run the code, etc...). When running the container it should expose the jupyter notebook on one port and expose a Flask API on another one. The Flask app contains two endpoints:
  - One for training the model
  - One for querying the last trained model with an audio file of our choice in the dataset
* A README.md which should contain the commands to build and run the docker container, as well as how to perform the queries to the API. 
* Any necessary .py, .sh or other files needed to run your code.

# Libraries Loading

In [16]:
!ls ..

assistproba  DataVis  DeepLearning  LabDS  OptforML  Visiumapp


In [1]:
import glob
from scipy.io import wavfile
import os
import numpy as np
import pandas as pd
import numpy as np
from flask import render_template
import traceback
import pickle
import pandas as pd
from IPython.core.debugger import set_trace

In [ ]:
render_template()

# Data Preparation & Cleaning

In [7]:
DE2EN = {'W':'A', #Wut-Anger
         'L':'B', #Langeweile-Bordom
         'E':'D', #Ekel-Disgust
         'A':'F', #Angst-Fear
         'F':'H', #Freude-Happiness
         'T':'S',
         'N':'N'} #Traueer-Sadness
EN2DE = {item:key for key,item in DE2EN.items()}

In [8]:
DATA_DIR = "./data"
WAV_DIR = os.path.join(DATA_DIR,"wav")
temp = wavfile.read("./data/wav/03a01Fa.wav")

In [9]:
def parse_filename(filename):
    """
    parses the attributes of a given sample based on its filename
    """
    speaker_id = int(filename[:2])
    text_id = filename[2:5]
    emotion_de = filename[5]
    emotion_en = DE2EN[emotion_de]
    return speaker_id,text_id,emotion_en

In [10]:
speaker_data = [[3  , 'male',  31],
                [8  , 'female',34 ],
                [9  , 'female',21 ],
                [10 , 'male',  32 ],
                [11 , 'male',  26 ],
                [12 , 'male',  30] ,
                [13 , 'female',32], 
                [14 , 'female',35] ,
                [15 , 'male',  25] ,
                [16 , 'female',31]]
speaker_data = pd.DataFrame(speaker_data,columns = ['speaker_id','sex','age'])
speaker_data.set_index('speaker_id',inplace=True)

In [11]:
def load_data(wav_dir=WAV_DIR):
    for root, dirs, files in os.walk(wav_dir, topdown=False):
        paths = [os.path.join(root,file) for file in files]
        #the sample rate is always the same, we discard it
        data = []
        for file in files:
            audio_data = wavfile.read(os.path.join(root,file))[1]
            speaker_id,text_id,emotion_en = parse_filename(file)
            row = [speaker_id,text_id,emotion_en,audio_data]
            data.append(row)
    res = pd.DataFrame(data,columns=["speaker_id","text_id","emotion","data"])
    return res.join(speaker_data,on="speaker_id")

In [12]:
load_data()

/home/ymentha/anaconda3/envs/ML/lib/python3.7/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


,speaker_id,text_id,emotion,data,sex,age
0,13,b03,A,"[12, -39, -39, -26, -45, -16, -45, -74, -88, -...",female,32
1,8,b02,A,"[0, -4, -2, 0, 0, -2, -2, -2, -2, -4, -6, -4, ...",female,34
2,13,a07,S,"[0, 36, -12, -60, -60, -72, -120, -144, -192, ...",female,32
3,3,b02,A,"[41, 37, 25, 7, -3, -1, 1, 14, 19, 19, 12, 1, ...",male,31
4,9,b02,S,"[0, 0, 0, -29, -58, -88, -58, -58, -58, -88, -...",female,21
...,...,...,...,...,...,...
530,3,a05,S,"[7, -23, 0, -7, -7, 0, -23, 0, -23, -7, -15, -...",male,31
531,16,b02,A,"[1, 1, 1, 1, 1, 0, -3, -3, -3, -3, -5, -5, -3,...",female,31
532,13,a05,B,"[69, 174, -75, -284, -203, -81, -110, -29, 58,...",female,32
533,8,a02,B,"[-6, -3, 6, 0, 3, 6, 0, 6, 6, 13, 13, 3, 13, 1...",female,34


# Feature Engineering & Modeling

In [13]:
def dummy_model(filename):
    data = wavfile.read(os.path.join(WAV_DIR,filename))[1]
    return filename[5]

In [14]:
dummy_model('03a01Fa.wav')

'F'

# Results & Visualizations

In [11]:
app = Flask(__name__)
@app.route('/')
def index():
    return render_template('./index.html')

In [43]:
# App definition
app = Flask(__name__,template_folder='templates')
 
# importing models
with open('webapp/model/model.pkl', 'rb') as f:
    classifier = pickle.load (f)

with open('webapp/model/model_columns.pkl', 'rb') as f:
    model_columns = pickle.load (f)
 
 
@app.route('/')
def welcome():
    return "Home Speech Recognition \n"


@app.route('/predict', methods=['GET','POST'])
def post():
  
    if flask.request.method == 'GET':
        return "Prediction page\n"
 
    if flask.request.method == 'POST':
        try: 
            #json_ is a string here
            json_ = request.json
            print(json_)
            #set_trace()
            pred = dummy_model(json_)
            return jsonify({
               "pred":str(pred)
           })
 
        except:
            return jsonify({
                    "FUCK"
               })